<a href="https://colab.research.google.com/github/ManuSinghYadav/Pneumonia-Classification-from-X-Rays/blob/main/Pneumonia_Classification_from_X_Rays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Loading the Dataset & Image Preprocessing
Dataset of Images can be downloaded from [here](https://www.kaggle.com/datasets/khoongweihao/covid19-xray-dataset-train-test-sets).

Loaded the images from the drive after mounting it in our colab.

In [26]:
import tensorflow as tf

Preprocessing of the training dataset.

In [27]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 148 images belonging to 2 classes.


Preprocessing of the testing dataset.

In [28]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 40 images belonging to 2 classes.


###Building & Training our CNN model

In [29]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [30]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [31]:
cnn.add(tf.keras.layers.Flatten())

In [32]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [33]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [34]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
5/5 [==============================] - 8s 2s/step - loss: 2.1912 - accuracy: 0.5068 - val_loss: 0.4607 - val_accuracy: 0.6250
Epoch 2/30
5/5 [==============================] - 6s 1s/step - loss: 0.8663 - accuracy: 0.5878 - val_loss: 0.3335 - val_accuracy: 0.9000
Epoch 3/30
5/5 [==============================] - 6s 1s/step - loss: 0.5927 - accuracy: 0.7162 - val_loss: 0.3335 - val_accuracy: 0.9000
Epoch 4/30
5/5 [==============================] - 7s 1s/step - loss: 0.4526 - accuracy: 0.7973 - val_loss: 0.1753 - val_accuracy: 0.9750
Epoch 5/30
5/5 [==============================] - 6s 1s/step - loss: 0.3574 - accuracy: 0.8581 - val_loss: 0.1364 - val_accuracy: 0.9750
Epoch 6/30
5/5 [==============================] - 6s 1s/step - loss: 0.3346 - accuracy: 0.8581 - val_loss: 0.1172 - val_accuracy: 0.9750
Epoch 7/30
5/5 [==============================] - 7s 2s/step - loss: 0.3384 - accuracy: 0.8649 - val_loss: 0.1430 - val_accuracy: 0.9500
Epoch 8/30
5/5 [=========================

Our CNN model is now trained and its efficiency can be seen from the validation accuracy, which is close to 100%. **But our main goal is higer precision (lower false positive) and higher recall (lower false negative)**.

###Testing our model seprately to calculate the precision and recall.

In [35]:
import numpy as np
from keras.preprocessing import image

In [36]:
#Here all the test images are of PNEUMONIA, so here we're calculating the "true-positive" and "false-negative"

import os

file_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test/PNEUMONIA')
img_name = []
flattened_features = []
tp = 0
fn = 0


for i in file_list:
  if i=='.ipynb_checkpoints':
    continue
  else:
    a = '/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test/PNEUMONIA/{}'.format(i)
    img = image.load_img(a, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    result = cnn.predict(img/255.0)
    if result[0][0] > 0.5:
      prediction = 'Pneumonia'
      tp = tp + 1
    else:
      prediction = 'Normal'
      fn = fn + 1

1/1 [==============================] - 0s 31ms/step


In [37]:
fn

0

In [38]:
#Here all the test images are NORMAL, so here we're calculating the "true-negative" and "false-positive"

import os
tn = 0
fp = 0

file_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test/NORMAL')
img_name = []
flattened_features = []


for i in file_list:
  if i=='.ipynb_checkpoints':
    continue
  else:
    a = '/content/drive/MyDrive/Colab Notebooks/Pneumonia Dataset/test/NORMAL/{}'.format(i)
    img = image.load_img(a, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    result = cnn.predict(img/255.0)
    if result[0][0] > 0.5:
      prediction = 'Pneumonia'
      fp = fp + 1
    else:
      prediction = 'Normal'
      tn = tn + 1

1/1 [==============================] - 0s 26ms/step


In [39]:
print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\nFalse Negative: {}".format(tp, tn, fp, fn))

True Positive: 20
True Negative: 20
False Positive: 0
False Negative: 0


In [40]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Precision: {}\nRecall: {}".format(precision, recall))

Precision: 1.0
Recall: 1.0
